# Notebook Goals
The goal of this notebook is to replicate the result of HOLC Chicago. Perhaps do the expansion job on the other python notebooks.

In [4]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context('paper')

paper_cmap = {"A":"#5FCE72","B":"#0BC0ED","C":"#FFD419","D":"#FF4B19"}
paper_colors = LinearSegmentedColormap('paper_colors', paper_cmap)

import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import glob
import requests

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely import wkt

import psycopg2  # (if it is postgres/postgis)
conn = psycopg2.connect(database="chicago_fha", user="wonyoungso", password="",
    host="localhost")
cursor = conn.cursor()


from sqlalchemy import create_engine
engine = create_engine('postgresql://wonyoungso:@localhost:5432/chicago_fha')

from geoalchemy2 import Geometry, WKTElement


import warnings
warnings.filterwarnings('ignore')


from patsylearn import PatsyModel, PatsyTransformer

import libpysal as ps

import statsmodels.api as sm
import statsmodels.stats as st


In [5]:
df_final = gpd.read_file('data/chicago_census_historical/chicago_census_historical.shp')

df_final = df_final.rename(columns={'asian': 'asian',
 'asian_pe_1': 'asian_perc_norm',
 'asian_perc': 'asian_perc',
 'black': 'black',
 'black_norm': 'black_norm',
 'black_pe_1': 'black_perc_norm',
 'black_perc': 'black_perc',
 'college': 'college',
 'college__1': 'college_perc_norm',
 'college_pe': 'college_perc',
 'gisjoin': 'gisjoin',
 'gisjoin_19': 'gisjoin_1940',
 'hispanic': 'hispanic',
 'hispanic_1': 'hispanic_perc_norm',
 'hispanic_p': 'hispanic_perc',
 'holc_grade': 'holc_grade',
 'homes': 'homes',
 'ln_homeval': 'ln_homeval_norm',
 'ln_income_': 'ln_income_norm',
 'ln_media_1': 'ln_median_homevalue_adj',
 'ln_median_': 'ln_median_income',
 'map_id': 'map_id',
 'median_h_1': 'median_homevalue_adj',
 'median_hom': 'median_homevalue',
 'median_i_1': 'median_income',
 'median_inc': 'median_income_adj',
 'other': 'other',
 'other_perc': 'other_perc',
 'owned_pe_1': 'owned_perc_norm',
 'owned_perc': 'owned_perc',
 'populati_1': 'population_density',
 'populati_2': 'population_density_norm',
 'populati_3': 'population_norm',
 'population': 'population',
 'unemploy_1': 'unemployed_perc',
 'unemploy_2': 'unemployed_perc_norm',
 'unemployed': 'unemployed',
 'white': 'white',
 'white_norm': 'white_norm',
 'white_pe_1': 'white_perc_norm',
 'white_perc': 'white_perc',
 'year': 'year'})


In [11]:
df_final['nonblack_perc'] = 1 - df_final['black_perc']/df_final['population']
df_final['perc_college_10plus'] = [1 if x > 0.10 else 0 for x in df_final['college_perc']]

# Need to create some extra features for the propensity scoring to increase similarity between 'before' tracts. 
df_final['perc_black_20plus'] = [1 if x > 0.20  else 0 for x in df_final['black_perc']]
df_final['perc_college_10plus'] = [1 if x > 0.10  else 0 for x in df_final['college_perc']]

# Create a point that represents roughly downtown is 
downtown = Point(-87.62, 41.855)
df_final['dist_downtown'] = df_final.distance(Point(-87.62,41.855))


Create different treatment periods

In [13]:
df_final[['year']].value_counts()

year
1940    789
1930    788
1970    787
2000    786
1960    785
1990    785
2010    785
1980    784
dtype: int64

In [14]:
df_final1 = df_final[df_final.year != 1950]
df_final1.loc[df_final1.year <= 1940,'period']='pre'
df_final1.loc[(df_final1.year > 1940) & (df_final1.year <= 1980), 'period']='post'

df_final1.loc[(df_final1.year>1980),'period']='reversal'

,year,period
0,1930,pre
1,1940,pre
2,1960,post
3,1970,post
4,1980,post
...,...,...
6284,1970,post
6285,1980,post
6286,1990,reversal
6287,2000,reversal
